In [1]:
import pandas as pd
import re
import os
import torch
import numpy as np

from transformers import BertModel, BertTokenizer

# test dataset

In [2]:
test_df = pd.read_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/test.csv')

# Bert model +  logistic for classification

In [3]:
class Model(torch.nn.Module):
    
    def __init__(self, ):
        
        super(Model, self).__init__()
        self.base_model = BertModel.from_pretrained('bert-base-uncase')
        self.fc1 = torch.nn.Linear(768,1)
        
    def forward(self, ids, masks):
        x = self.base_model(ids, attention_mask = masks)[1]
        x = self.fc1(x)
        return x

In [6]:
path_to_model = 'C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/'

In [8]:
model = torch.load(os.path.join(path_to_model, 'model.pth'))

In [19]:
device ='cpu'

In [20]:
model = model.to(device)

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
def bert_encode(text, max_len=512):
    
    text = tokenizer.tokenize(text)
    text = text[:max_len-2]
    input_sequence = ["[CLS]"] + text + ["[SEP]"]
    tokens = tokenizer.convert_tokens_to_ids(input_sequence)
    tokens += [0] * (max_len - len(input_sequence))
    pad_masks = [1] * len(input_sequence) + [0] * (max_len - len(input_sequence))
    
    return tokens, pad_masks

In [14]:
class TestDataset(torch.utils.data.Dataset):
    
    def __init__(self, test_tokens, test_pad_masks):
        
        super(TestDataset, self).__init__()
        self.test_tokens = test_tokens
        self.test_pad_masks = test_pad_masks
        
    def __getitem__(self, index):
        
        tokens = self.test_tokens[index]
        masks = self.test_pad_masks[index]
        
        return (tokens, masks)
    
    def __len__(self,):
        
        return len(self.test_tokens)

In [15]:
test_tokens = []
test_pad_masks = []
for text in test_df.text:
    tokens, masks = bert_encode(text)
    test_tokens.append(tokens)
    test_pad_masks.append(masks)
    
test_tokens = np.array(test_tokens)
test_pad_masks = np.array(test_pad_masks)

In [16]:
test_dataset = TestDataset(
    test_tokens=test_tokens,
    test_pad_masks=test_pad_masks
)

In [17]:
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=3, shuffle=False)

In [21]:
model.eval()
y_preds = []
for (tokens, masks) in test_dataloader:

    y_pred = model(
                torch.tensor(tokens, dtype=torch.long).to(device),
                torch.tensor(masks, dtype=torch.long).to(device),
            )
    y_preds += y_pred.detach().cpu().numpy().squeeze().tolist()

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
